# Install Dependencies

In [1]:
%%capture
!pip install llama-index-core pymupdf llama-index chromadb sentence-transformers pydantic llama-index-llms-openai llama-index-embeddings-huggingface llama-index-vector-stores-chroma
!pip install llama-index-readers-file
!pip install llama-index-embeddings-azure-openai
!pip install llama-index-llms-azure-openai

In [22]:
# สร้าง retriever
import chromadb
import os
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI
from llama_index.core import StorageContext
from llama_index.core import VectorStoreIndex
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core.node_parser import TokenTextSplitter
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core.retrievers import BaseRetriever
from llama_index.core import PromptTemplate

# นำเข้าไฟล์ .env 
from dotenv import load_dotenv
load_dotenv()

PDF_DATA_PATH="../data/personal_data_protection_policy.pdf"
EMBEDDING_MODEL="BAAI/bge-base-en-v1.5"
CHUNK_SIZE=1000
CHUNK_OVERLAP=250
OPENAI_TEMP=0
CHROMA_PERSISTENCE_PATH="./chroma_db"
CHROMA_COLLECTION_NAME="test4543ss332"
RETRIEVER_TOP_K=5

# Setup LLM & Embeddings

In [ ]:
embed_model = OpenAIEmbedding(model="text-embedding-3-large")

embeddings = embed_model.get_text_embedding(
    "Open AI new Embeddings models is great."
)
print(embeddings)

In [17]:
llm = OpenAI(model="gpt-4o")

# ลองใช้งาน LLM
llm.complete("What is the capital of Thailand?")

CompletionResponse(text='The capital of Thailand is Bangkok.', additional_kwargs={}, raw=ChatCompletion(id='chatcmpl-A5aY3Ob5EOcm3JttdRTfQoOgaM9eX', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The capital of Thailand is Bangkok.', refusal=None, role='assistant', function_call=None, tool_calls=None))], created=1725895495, model='gpt-4o-2024-05-13', object='chat.completion', service_tier=None, system_fingerprint='fp_25624ae3a5', usage=CompletionUsage(completion_tokens=7, prompt_tokens=14, total_tokens=21)), logprobs=None, delta=None)

# Setup LlamaIndex Embeddings and Chunking

In [24]:
splitter = TokenTextSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap=CHUNK_OVERLAP,
)

Settings.node_parser = splitter

Settings.embed_model = embed_model

Settings.chunk_size=CHUNK_SIZE
Settings.chunk_overlap=CHUNK_OVERLAP

Settings.llm = LLM

# Read PDFs

In [10]:
from llama_index.readers.file.docs import PDFReader

reader = PDFReader()

documents = reader.load_data(PDF_DATA_PATH)

In [ ]:
# check documents
docs = [doc.text for doc in documents]
print("Document's length: ", len(docs))
print("\n".join(docs))

print(documents)

# Setup Chroma Vector Store and Ingest Documents

In [34]:
def generate_retriever(documents) -> VectorStoreIndex:
    db = chromadb.Client()
    
    # [OPTIONAL] delete if existing
    try:
        db.delete_collection(CHROMA_COLLECTION_NAME)
    except ValueError:
        print("[ChromaDB] Failed to delete collection.")
        
    chroma_collection = db.create_collection(CHROMA_COLLECTION_NAME)
    
    vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
    
    # StorageContext is not ded like the other one
    storage_context = StorageContext.from_defaults(vector_store=vector_store)
    
    nodes = splitter.get_nodes_from_documents(documents)
    
    index = VectorStoreIndex(nodes, storage_context=storage_context, show_progress=True)

    return index

vector_store_index = generate_retriever(documents) 
retriever = vector_store_index.as_retriever(similarity_top_k=RETRIEVER_TOP_K)

Generating embeddings:   0%|          | 0/17 [00:00<?, ?it/s]

Generating embeddings: 100%|██████████| 17/17 [00:01<00:00,  9.94it/s]


In [35]:
# Sanity check for docs in the vDB
res = retriever.retrieve("การเก็บรวบรวมข้อมูลส่วนบุคคล มีแนวทางอย่างไรบ้าง?")

In [36]:
print(f"Retrieve {len(res)} chunks")

Retrieve 5 chunks


# Testing our RAG workflow

In [37]:
query_engine = vector_store_index.as_query_engine()

In [38]:
user_query = "การเก็บรวบรวมข้อมูลส่วนบุคคล มีแนวทางอย่างไรบ้าง?"
response = query_engine.query(user_query)

In [ ]:
print(response)

## Using the prompt-template

ทดลองใช้  Template เพื่อให้เราได้คำตอบในรูปแบบที่เราต้องการ

In [41]:
QUERY_PROMPT_TEMPLATE = """
# Task
From given document answer the following question.

# Instructions
- Answer the following question based on the given document.
- You need to cite the source of the answer from the document.

# Document:
{document}

# Question:
{question}

# Answer:
"""

In [48]:
def get_answer_from_rag(
    retriever: BaseRetriever,
    llm: OpenAI,
    qa_prompt: PromptTemplate,
    question: str,
) -> str:
    # Retrieve the chunks
    chunks = retriever.retrieve(question)
    response = llm.complete(
        qa_prompt.format(
            document=chunks,
            question=question,
        )
    )
    
    return response


In [49]:
answer = get_answer_from_rag(
    retriever=query_engine,
    llm=LLM,
    qa_prompt=QUERY_PROMPT_TEMPLATE,
    question="การเก็บรวบรวมข้อมูลส่วนบุคคล มีแนวทางอย่างไรบ้าง?",
)

In [50]:
print("Answer: ", answer)

Answer:  การเก็บรวบรวมข้อมูลส่วนบุคคลมีแนวทางดังนี้:

1. เมื่อพ้นระยะเวลาการใช้งานตามวัตถุประสงค์ในการเก็บรวบรวมข้อมูลส่วนบุคคล
2. เมื่อเกินความจำเป็น
3. เมื่อเจ้าของข้อมูลส่วนบุคคลร้องขอ
4. เมื่อเจ้าของข้อมูลได้ถอนความยินยอม

เว้นแต่หน่วยงานหรือผู้รับผิดชอบจะสามารถเก็บรักษาข้อมูลส่วนบุคคลไว้ต่อไปตามที่กฎหมายแต่ละประเทศกำหนด

(ที่มา: personal_data_protection_policy.pdf, หน้า 8)
